# **Introduction**
---
This project seeks to leverage the power of computer vision and machine learning to perform image segmentation on a car's point of view while on the road. The goal is to accurately analyze the captured visual data and provide a more comprehensive understanding of the environment in which the vehicle is traveling. The insights gained from this analysis will be crucial in improving the safety and efficiency of transportation systems, particularly in the development of autonomous vehicles. Ultimately, the project aims to contribute to the advancement of technology that will enhance our ability to navigate our world safely and efficiently.

## **Table of Contents**
---
* [Video Information Extraction](#Video-Information-Extraction)
* [Vehicle POV Segmentation](#Vehicle-POV-Segmentation)
* [Usage](#Usage)

In [1]:
# Importing required libraries
from IPython import display
from ultralytics import YOLO
import supervision as sv
from datetime import timedelta
import os
import cv2

## **Video Information Extraction**
---

In [2]:
def get_video_info(video_path):
    
    # Extracting information about the video
    video_info = sv.VideoInfo.from_video_path(video_path)
    width, height, fps, total_frames = video_info.width, video_info.height, video_info.fps, video_info.total_frames
    
    # Calculating the length of the video by dividing the total number of frames by the frame rate and rounding to the nearest second
    video_length = timedelta(seconds = round(total_frames / fps))
    
    # Print out the video resolution, fps, and length
    print(f"\033[1mVideo Resolution:\033[0m ({width}, {height})")
    print(f"\033[1mFPS:\033[0m {fps}")
    print(f"\033[1mLength:\033[0m {video_length}")

In [3]:
# Extracting information of the test video
get_video_info('Data/Test Video/video2.mp4')

Video Resolution: (3840, 2160)
FPS: 59
Length: 0:00:27


## **Vehicle POV Segmentation**
---

In [4]:
def vehicle_pov_segment(source_path, destination_path):

  # Load the pre-trained YOLO model
  model = YOLO('yolov8x-seg.pt')

  # Extract information about the video from the given source path
  video_info = sv.VideoInfo.from_video_path(source_path)

  # Create a video out path by combining the destination path and the video name
  video_name = os.path.splitext(os.path.basename(source_path))[0] + ".mp4"
  video_out_path = os.path.join(destination_path, video_name)

  # Create a video writer object for the output video
  video_out = cv2.VideoWriter(video_out_path, cv2.VideoWriter_fourcc(*'mp4v'), video_info.fps, (video_info.width, video_info.height))

  # Loop over each frame of the video and perform object detection and tracking
  for result in model.track(source = source_path, tracker = 'bytetrack.yaml', stream=True, agnostic_nms=True):
  
    # Plotting masks only 
    frame = result.plot(conf = False,
                        labels = False,
                        boxes = False,
                        masks = True,
                        probs = False)
    
    # Write the frame to the output video
    video_out.write(frame)

  # Release the video writer and clear the Jupyter Notebook output  
  video_out.release()
  display.clear_output()

## **Usage**
---

In [5]:
# Performing vehicle's POV segmentation
vehicle_pov_segment("Data/Test Video/video2.mp4",
                "Data/Test Result")